In [22]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *

from learning_utilities import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

Setting environment for Gordon
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
stack = 'MD594'

In [24]:
labelings, _, _ = DataManager.load_annotation_v2(stack=stack, username='yuncong', orientation='sagittal', downsample=1,
                                                 annotation_rootdir=annotation_midbrainIncluded_v2_rootdir)

Loaded annotation /oasis/projects/nsf/csd395/yuncong/CSHL_labelings_v2/MD594/MD594_sagittal_downsample1_yuncong_08212016032024.pkl.


In [25]:
labelings2, _, _ = DataManager.load_annotation_v2(stack=stack, username='yuncong', orientation='sagittal', downsample=32,
                                                 annotation_rootdir=annotation_midbrainIncluded_v2_rootdir)

Loaded annotation /oasis/projects/nsf/csd395/yuncong/CSHL_labelings_v2/MD594/MD594_sagittal_downsample32_yuncong_08182016084049.pkl.


In [26]:
import uuid

CONTOUR_IS_INTERPOLATED = 1

contour_entries = {}

for sec, contours in labelings['polygons'].iteritems():
    for contour in contours:        
        contour_id = str(uuid.uuid4().fields[-1])
        
        contour_entry = {'name': contour['label'],
            'label_position': contour['labelPos'] if 'labelPos' in contour else None,
           'side': contour['side'] if 'side' in contour else None,
           'creator': 'yuncong',
           'time_created': labelings['timestamp'],
            'edits': [],
#            'edits': [{'user': 'Lauren', 'timestamp': 2016}],
            'vertices': contour['vertices'],
            'downsample': 1,
#            'vertices': {'downsample1': vertices},
#             'vertices': Series([vertices], index=['downsample1']),
           'flags': CONTOUR_IS_INTERPOLATED if 'type' in contour and contour['type'] == 'interpolated' else 0,
            'section': sec,
            'position': None,
            'orientation': 'sagittal',
            'parent_structure': [],
            'id': contour_id}
        
#     contour_entries.append(contour_entry)
        assert contour_id not in contour_entries
        contour_entries[contour_id] = contour_entry
    
# pickle.dump(contour_entries, open('annotation_v3.pkl', 'w'))

In [27]:
for sec, contours in labelings2['polygons'].iteritems():
    for contour in contours:        
        contour_id = str(uuid.uuid4().fields[-1])
        
        contour_entry = {'name': contour['label'],
            'label_position': contour['labelPos'] if 'labelPos' in contour else None,
           'side': contour['side'] if 'side' in contour else None,
           'creator': 'yuncong',
           'time_created': labelings['timestamp'],
            'edits': [],
#            'edits': [{'user': 'Lauren', 'timestamp': 2016}],
            'vertices': contour['vertices'],
            'downsample': 32,
#            'vertices': {'downsample1': vertices},
#             'vertices': Series([vertices], index=['downsample1']),
           'flags': CONTOUR_IS_INTERPOLATED if 'type' in contour and contour['type'] == 'interpolated' else 0,
            'section': sec,
            'position': None,
            'orientation': 'sagittal',
            'parent_structure': [],
            'id': contour_id}
        
#     contour_entries.append(contour_entry)
        assert contour_id not in contour_entries
        contour_entries[contour_id] = contour_entry
    
# pickle.dump(contour_entries, open('annotation_v3.pkl', 'w'))

In [28]:
len(contour_entries)

1893

In [29]:
from pandas import *

In [30]:
df = DataFrame(contour_entries).T

In [31]:
! rm /home/yuncong/csd395/CSHL_labelings_v2/{stack}_annotation_v3.h5

rm: cannot remove `/home/yuncong/csd395/CSHL_labelings_v2/MD594_annotation_v3.h5': No such file or directory


In [32]:
df.to_hdf('/home/yuncong/csd395/CSHL_labelings_v2/%(stack)s_annotation_v3.h5' % {'stack': stack}, 'contours')

In [ ]:
df.to_hdf('/home/yuncong/csd395/CSHL_labelings_v2/%(stack)s_annotation_v3.h5' % {'stack': stack}, 'structures')

In [ ]:
contours = df[df['orientation'] == 'sagittal']

In [ ]:
grouped = contours.groupby('section')
for sec, group in grouped:
    for cid, contour in group.iterrows():
        print cid, contour[]

In [ ]:
group.iloc[0].index

In [ ]:
for c_id, cc in c.iterrows():
    print c_id, cc['section'], cc['vertices'].shape

In [ ]:
contours = read_hdf('/home/yuncong/csd395/CSHL_labelings_v2/%(stack)s_annotation_v3.h5' % {'stack': stack})

In [ ]:
from pandas import *

In [ ]:
def get_dtype(data):
    """Given a dict, generate a nested numpy dtype"""
    fields = []
    for (key, value) in data.items():
        # make strings go to the next 64 character boundary
        # pytables requires an 8 character boundary
        if isinstance(value, unicode):
            value += u' ' * (64 - (len(value) % 64))
            # pytables does not support unicode
            if isinstance(value, unicode):
                value = value.encode('utf-8')
        elif isinstance(value, str):
            value += ' ' * (64 - (len(value) % 64))

        if isinstance(value, dict):
            fields.append((key, get_dtype(value)))
        else:
            value = np.array(value)
            fields.append((key, '%s%s' % (value.shape, value.dtype)))
    return np.dtype(fields)

In [ ]:
get_dtype(contour_entry)

In [ ]:
def _recurse_row(row, base, data):
    for (key, value) in data.items():
        new = base + key
        if isinstance(value, dict):
            _recurse_row(row, new + '/', value)
        else:
            row[new] = value

In [ ]:
def add_row(tbl, data):
    """Add a new row to a table based on the contents of a dict.
    """
    row = tbl.row
    for (key, value) in data.items():
        if isinstance(value, dict):
            _recurse_row(row, key + '/', value)
        else:
            row[key] = value
    row.append()
    tbl.flush()

In [ ]:
df = DataFrame(contour_entries)

In [ ]:
df.loc[0]

In [ ]:
fid = tables.open_file('annotation_v3.h5', 'w')

In [ ]:
fid.remove_node('/contours')

In [ ]:
tbl = fid.create_table('/', 'contours', get_dtype(contour_entry))

In [ ]:
for contour_entry in contour_entries:
    add_row(tbl, contour_entry)

In [ ]:
row.dtype

In [ ]:
# https://gist.github.com/blink1073/feb3ac5920653ae222c2
[dict(zip(tbl.dtype.names, row)) for row in tbl.read_where(('orientation == b"sagittal"'))]

In [ ]:
store = HDFStore('annotation_v3.h5')

In [ ]:
store.get_node('contours')

In [ ]:
store.close()

In [ ]:
add_row(store.get_node('contours'), contour_entry)

In [ ]:
store

In [ ]:
store.create_table_index('contours')

In [ ]:
store.put('contours', df, format='table')

In [ ]:
        
    structure_entry = {'name': convert_name_to_unsided(name_s),
       'side': extract_side_from_name(name_s),
       'creator': 'yuncong',
       'time_created': None,
       'edits': [('Lauren', None)],
       'contours': vertices,
         'bbox': {'downsample1': (100,200,100,200,100,200), 'downsample32': None},
       'volume': {'downsample1': np.random.randint(0, 1, (100,100,100)).astype(np.bool),
                 'downsample32': None},
       'flag': CONTOUR_IS_INTERPOLATED,
        'section': sec}


In [ ]:
w1 = {'name': 'Tz',
           'side': 'L',
           'creator': 'yuncong',
           'time_created': '2016',
           'edits': [('Lauren', '20160112')],
           'contours': [(4,2,76,8,9)],
             'bbox': [{'downsample1': (100,200,100,200,100,200), 'downsample32': None}],
           'volume': {'downsample32': None,
                     'downsample1': np.random.randint(0, 1, (100,100,100)).astype(np.bool)},
           'flag': CONTOUR_IS_INTERPOLATED
    }

In [ ]:
w2 = {'name': 'Tz',
           'side': 'R',
           'creator': 'yuncong',
           'time_created': '2016',
           'edits': [('Lauren', '20160112')],
           'contours': [(4,2,76,8,9)],
             'bbox': [{'downsample1': (100,200,100,200,100,200), 'downsample32': None}],
           'volume': {'downsample32': None,
                     'downsample1': np.random.randint(0, 1, (100,100,100)).astype(np.bool)},
           'flag': CONTOUR_IS_INTERPOLATED
    }

In [ ]:
w = [w1] + [w2]

In [ ]:
q = DataFrame(w, index=range(len(w)))

In [ ]:
q.index

In [ ]:
store = HDFStore('annotation_v3.h5')

In [ ]:
store

In [ ]:
q.to_hdf('annotation_v3.h5', '3d_structures', dropna=True)

In [ ]:
q.to_hdf('annotation_v3.h5', '3d_structures', dropna=True)

In [ ]:
store

In [ ]:
d = pandas.read_hdf('annotation_v3.h5')